In [6]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from fractions import Fraction
from math import gcd
from numpy.random import choice
from scipy.stats import entropy

from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram, circuit_drawer
from IPython.display import display

# Create output directory to store results
output_dir = "shorSim"
os.makedirs(output_dir, exist_ok=True)

# Function to build the inverse Quantum Fourier Transform circuit
def qft_dagger(n):
    qc = QuantumCircuit(n)
    for qubit in range(n // 2):
        qc.swap(qubit, n - qubit - 1)
    for j in range(n):
        for m in range(j):
            qc.cp(-np.pi / float(2 ** (j - m)), m, j)
        qc.h(j)
    return qc

# Function to generate the modular exponentiation circuit a^x mod 15 for selected values of a
def c_amod15(a, power):
    a = int(a)
    if a not in [2, 7, 8, 11, 13]:
        raise ValueError("Only a in [2, 7, 8, 11, 13] is supported for N=15.")
    U = QuantumCircuit(4)
    for _ in range(power):
        if a in [2, 13]:
            U.swap(0, 1); U.swap(1, 2); U.swap(2, 3)
        elif a in [7, 8]:
            U.swap(2, 3); U.swap(1, 2); U.swap(0, 1)
        elif a == 11:
            U.swap(2, 3); U.swap(0, 2)
        if a in [7, 11, 13]:
            for q in range(4): U.x(q)
    return U.to_gate().control()

# Run a single trial of quantum phase estimation simulation using AerSimulator
def run_qpe_trial(a, n_count=8, shots=1024):
    a = int(a)
    qc = QuantumCircuit(n_count + 4, n_count)
    
    # Apply Hadamard gates to counting register
    for q in range(n_count):
        qc.h(q)
    # Initialize target register
    qc.x(n_count + 3)
    
    # Apply controlled-U operations
    for q in range(n_count):
        qc.append(c_amod15(a, 2 ** q), [q] + [i + n_count for i in range(4)])
    
    # Apply inverse QFT and measure
    qc.append(qft_dagger(n_count), range(n_count))
    qc.measure(range(n_count), range(n_count))

    sim = AerSimulator()
    tqc = transpile(qc, sim)
    result = sim.run(tqc, shots=shots).result()
    counts = result.get_counts()

    # Analyze and decode measurement results
    rows = []
    for bitstring, count in counts.items():
        decimal = int(bitstring, 2)
        phase = decimal / (2 ** n_count)
        frac = Fraction(phase).limit_denominator(15)
        r = int(frac.denominator)
        rows.append([bitstring.zfill(n_count), count, f"{phase:.4f}", f"{frac.numerator}/{frac.denominator}", r])

    df = pd.DataFrame(rows, columns=["Bitstring", "Counts", "Phase", "Fraction", "Candidate r"])
    return df, counts, qc

# Try factoring N using the guessed order r
def try_factorization(a, N, r):
    a, N, r = int(a), int(N), int(r)
    if r % 2 != 0:
        print(f"  Rejected: r = {r} is not even.")
        return None
    if pow(a, r, N) != 1:
        print(f"  Rejected: a^{r} mod N ≠ 1")
        return None
    x1 = pow(a, r // 2) - 1
    x2 = pow(a, r // 2) + 1
    guess1 = gcd(x1, N)
    guess2 = gcd(x2, N)
    print(f"  Trying gcd(a^{r//2} - 1, N) = gcd({x1}, {N}) = {guess1}")
    print(f"  Trying gcd(a^{r//2} + 1, N) = gcd({x2}, {N}) = {guess2}")
    if 1 < guess1 < N:
        print(f"  Success: Nontrivial factor found: {guess1}")
        return guess1
    if 1 < guess2 < N:
        print(f"  Success: Nontrivial factor found: {guess2}")
        return guess2
    print("  Failed: No nontrivial factor found with this r.")
    return None

# Experiment setup parameters
N = 15
n_count = 8
valid_a_values = [2, 7, 8, 11, 13]
shots_list = [4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048]

results = []
summary_rows = []

# Iterate over different shot values
for shots in shots_list:
    print(f"\n=== Running trials with shots = {shots} ===")
    trial_attempts = []
    success_count = 0
    phase_resolutions, peak_confidences, entropies = [], [], []

    # Run 3 trials per shot setting
    for trial in range(3):
        a = int(choice(valid_a_values))
        print(f"\nTrial {trial+1}: Trying a = {a}")
        df, counts, qc = run_qpe_trial(a, n_count, shots)

        print("\nPhase Estimation Result Table:")
        display(df)

        print("\nMeasurement Histogram:")
        fig = plot_histogram(counts, title=f"Histogram (a={a}, shots={shots})")
        histogram_path = f"{output_dir}/histogram_shots{shots}_trial{trial+1}.png"
        fig.savefig(histogram_path)
        plt.close(fig)

        # Attempt factoring using top 2 candidate r values only
        df_sorted = df.sort_values(by="Counts", ascending=False).head(2)
        tried_r = set()
        success = False
        for _, row in df_sorted.iterrows():
            r = int(row["Candidate r"])
            if r in tried_r: continue
            tried_r.add(r)
            print(f"  Trying candidate r = {r} from bitstring {row['Bitstring']} (phase ≈ {row['Phase']})")
            if try_factorization(a, N, r):
                print(f"  Trial {trial+1}: Success with a = {a}, r = {r}")
                success = True
                break
        if success:
            success_count += 1
        else:
            print(f"  Trial {trial+1}: Failed to factor using top 2 candidates.")

        trial_attempts.append(1)  # Always 1, since only 2 candidates tried max

        # Compute trial metrics
        df_all_sorted = df.sort_values(by="Counts", ascending=False)
        top_phases = df_all_sorted["Phase"].astype(float).values[:2]
        phase_res = abs(top_phases[0] - top_phases[1]) if len(top_phases) > 1 else 0.0
        total_counts = sum(counts.values())
        peak_conf = max(counts.values()) / total_counts
        probs = np.array(list(counts.values())) / total_counts
        dist_entropy = entropy(probs)

        phase_resolutions.append(phase_res)
        peak_confidences.append(peak_conf)
        entropies.append(dist_entropy)

    # Aggregate results across the 3 trials
    avg_attempt = np.mean(trial_attempts)
    success_rate = success_count / 3
    failure_rate = 1 - success_rate
    avg_phase_res = np.mean(phase_resolutions)
    avg_peak_conf = np.mean(peak_confidences)
    avg_entropy = np.mean(entropies)

    results.append((shots, avg_attempt))
    summary_rows.append((shots, avg_attempt, success_rate, failure_rate, avg_phase_res, avg_peak_conf, avg_entropy))

# Save summary results to CSV
summary_df = pd.DataFrame(summary_rows, columns=[
    "Shots", "Average Attempts", "Success Rate", "Failure Rate",
    "Avg Phase Resolution", "Avg Peak Confidence", "Avg Entropy"
])
summary_csv_path = os.path.join(output_dir, "shor_simulation_extended_summary.csv")
summary_df.to_csv(summary_csv_path, index=False)

# Generate and save graphs for each metric
metrics = [
    ("Average Attempts", "o"),
    ("Success Rate", "s"),
    ("Failure Rate", "x"),
    ("Avg Phase Resolution", "D"),
    ("Avg Peak Confidence", "^"),
    ("Avg Entropy", "v")
]

for metric, marker in metrics:
    plt.figure(figsize=(8, 5))
    plt.plot(summary_df["Shots"], summary_df[metric], marker=marker)
    plt.title(f"{metric} vs Shots (Aer Simulation)")
    plt.xlabel("Shots")
    plt.ylabel(metric)
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"{metric.replace(' ', '_')}_vs_Shots.png"))
    plt.close()

print(f"\nAll results and graphs have been saved in the '{output_dir}' directory.")



=== Running trials with shots = 4 ===

Trial 1: Trying a = 8

Phase Estimation Result Table:


,Bitstring,Counts,Phase,Fraction,Candidate r
0,00000000,2,0.0000,0/1,1
1,11000000,1,0.7500,3/4,4
2,01000000,1,0.2500,1/4,4



Measurement Histogram:
  Trying candidate r = 1 from bitstring 00000000 (phase ≈ 0.0000)
  Rejected: r = 1 is not even.
  Trying candidate r = 4 from bitstring 11000000 (phase ≈ 0.7500)
  Trying gcd(a^2 - 1, N) = gcd(63, 15) = 3
  Trying gcd(a^2 + 1, N) = gcd(65, 15) = 5
  Success: Nontrivial factor found: 3
  Trial 1: Success with a = 8, r = 4

Trial 2: Trying a = 11

Phase Estimation Result Table:


,Bitstring,Counts,Phase,Fraction,Candidate r
0,10101011,1,0.6680,2/3,3
1,10000000,1,0.5000,1/2,2
2,00101010,2,0.1641,1/6,6



Measurement Histogram:
  Trying candidate r = 6 from bitstring 00101010 (phase ≈ 0.1641)
  Trying gcd(a^3 - 1, N) = gcd(1330, 15) = 5
  Trying gcd(a^3 + 1, N) = gcd(1332, 15) = 3
  Success: Nontrivial factor found: 5
  Trial 2: Success with a = 11, r = 6

Trial 3: Trying a = 2

Phase Estimation Result Table:


...


Measurement Histogram:
  Trying candidate r = 4 from bitstring 11000000 (phase ≈ 0.7500)
  Trying gcd(a^2 - 1, N) = gcd(48, 15) = 3
  Trying gcd(a^2 + 1, N) = gcd(50, 15) = 5
  Success: Nontrivial factor found: 3
  Trial 1: Success with a = 7, r = 4

Trial 2: Trying a = 7

Phase Estimation Result Table:


,Bitstring,Counts,Phase,Fraction,Candidate r
0,00000000,495,0.0000,0/1,1
1,01000000,495,0.2500,1/4,4
2,10000000,535,0.5000,1/2,2
3,11000000,523,0.7500,3/4,4



Measurement Histogram:
  Trying candidate r = 2 from bitstring 10000000 (phase ≈ 0.5000)
  Rejected: a^2 mod N ≠ 1
  Trying candidate r = 4 from bitstring 11000000 (phase ≈ 0.7500)
  Trying gcd(a^2 - 1, N) = gcd(48, 15) = 3
  Trying gcd(a^2 + 1, N) = gcd(50, 15) = 5
  Success: Nontrivial factor found: 3
  Trial 2: Success with a = 7, r = 4

Trial 3: Trying a = 13

Phase Estimation Result Table:


,Bitstring,Counts,Phase,Fraction,Candidate r
0,00000000,530,0.0000,0/1,1
1,11000000,490,0.7500,3/4,4
2,10000000,508,0.5000,1/2,2
3,01000000,520,0.2500,1/4,4



Measurement Histogram:
  Trying candidate r = 1 from bitstring 00000000 (phase ≈ 0.0000)
  Rejected: r = 1 is not even.
  Trying candidate r = 4 from bitstring 01000000 (phase ≈ 0.2500)
  Trying gcd(a^2 - 1, N) = gcd(168, 15) = 3
  Trying gcd(a^2 + 1, N) = gcd(170, 15) = 5
  Success: Nontrivial factor found: 3
  Trial 3: Success with a = 13, r = 4

All results and graphs have been saved in the 'shorSim' directory.
